In [13]:
def maximize_coupons(item_prices, coupons):
    """
    Splits items into batches to maximize the total benefit from coupons with different thresholds and benefits.
    
    Parameters:
    - item_prices: List of item prices (list of floats or ints).
    - coupons: List of coupon dictionaries, each with 'threshold' and 'benefit'.
    
    Returns:
    - batches: List of batches, where each batch is a dictionary with keys:
        - 'items': List of item prices in the batch.
        - 'total': Total price of the batch.
        - 'coupon_threshold': The threshold of the coupon applied to the batch (if any).
        - 'coupon_benefit': The benefit of the coupon applied to the batch (if any).
    """
    # Calculate benefit-to-threshold ratio for each coupon
    for coupon in coupons:
        coupon['ratio'] = coupon['benefit'] / coupon['threshold']
    
    # Sort coupons based on benefit-to-threshold ratio in descending order
    coupons = sorted(coupons, key=lambda x: x['ratio'], reverse=True)
    
    # Sort items in ascending order to create batches that just meet the threshold
    items = sorted(item_prices)
    batches = []
    
    while items:
        batch = []
        total = 0
        coupon_applied = None
        coupon_benefit = 0
        
        # Attempt to apply the most beneficial coupon
        for coupon in coupons:
            threshold = coupon['threshold']
            benefit = coupon['benefit']
            temp_batch = []
            temp_total = 0
            items_to_remove = []
            
            # Greedily add smallest items until threshold is met or exceeded
            for item in items:
                temp_batch.append(item)
                temp_total += item
                items_to_remove.append(item)
                if temp_total >= threshold:
                    break  # Threshold met or exceeded
                    
            if temp_total >= threshold:
                # Apply coupon
                batch = temp_batch
                total = temp_total
                coupon_applied = threshold
                coupon_benefit = benefit
                # Remove used items from items
                for used_item in items_to_remove:
                    items.remove(used_item)
                break  # Move to next batch after applying coupon

        if not batch:
            # Can't apply any coupon, create a batch with remaining items
            batch = items.copy()
            total = sum(batch)
            coupon_applied = None
            coupon_benefit = 0
            items = []  # All items are now allocated

        batches.append({
            'items': batch,
            'total': total,
            'coupon_threshold': coupon_applied,
            'coupon_benefit': coupon_benefit
        })
    
    return batches

# Usage:

item_prices = [94.90, 7.21, 189.24, 4.64, 30.5, 9.68, 29.35, 2.04, 3.50, 6.64, 6.57]
coupons = [
    {'threshold': 169, 'benefit': 25},
    {'threshold': 89, 'benefit': 12},
    {'threshold': 39, 'benefit': 5},
]

batches = maximize_coupons(item_prices, coupons)

print("Batches:")
total_benefit = 0
for i, batch in enumerate(batches, 1):
    if batch['coupon_threshold']:
        coupon_info = f"Coupon Applied: Threshold = {batch['coupon_threshold']}, Benefit = {batch['coupon_benefit']}"
    else:
        coupon_info = "No Coupon Applied"
    total_benefit += batch['coupon_benefit']
    print(f"Batch {i}: Items = {batch['items']}, Total = {batch['total']:.2f}, {coupon_info}")
print(f"Total Benefit: {total_benefit}")


Batches:
Batch 1: Items = [2.04, 3.5, 4.64, 6.57, 6.64, 7.21, 9.68, 29.35, 30.5, 94.9], Total = 195.03, Coupon Applied: Threshold = 169, Benefit = 25
Batch 2: Items = [189.24], Total = 189.24, Coupon Applied: Threshold = 169, Benefit = 25
Total Benefit: 50
